### Context Managers in Python

In [ ]:
try:
    10 / 2
except ZeroDivisionError:
    print('Zero division exception occurred')
finally:
    print('finally ran!')    

finally ran!


In [ ]:
try:
    1 / 0
except ZeroDivisionError:
    print('Zero division exception occurred')
finally:
    print('finally ran!')

Zero division exception occurred
finally ran!


In [ ]:
def my_func():
    try:
        1/0
    except:
        print("We came here")
        return
    finally:
        print('finally running...')

my_func()

We came here
finally running...


In [ ]:
try:
    f = open('test.txt', 'w')
    a = 1 / 0
except:
    print('an exception occurred...')
finally:
    print('Closing file...')
    f.close()

an exception occurred...
Closing file...


Let's talk about context managers, and the pattern we are trying to solve:

1. Run some code to create some object(s)
2. Work with object(s)
3. Run some code when done to clean up object(s)

In [ ]:
with open('test.txt', 'w') as mangoose:
    print('inside with: mangoose closed?', mangoose.closed)
print('after with: mangoose closed?', mangoose.closed)

inside with: mangoose closed? False
after with: mangoose closed? True


In [ ]:
def test():
    with open('test.txt', 'w') as file:
        print('inside with: file closed?', file.closed)
        return file

file = test()

file.closed

inside with: file closed? False


True

In [ ]:
with open('test.txt', 'w') as f:
    print('inside with: file closed?', f.closed)
    raise ValueError()


inside with: file closed? False


ValueError: 

In [ ]:
f

<_io.TextIOWrapper name='test.txt' mode='w' encoding='UTF-8'>

In [ ]:
print('after with: file closed?', f.closed)

after with: file closed? True


Context managers can be used for more than just opening and closing files.

If we think about it there are two phases to a context manager:
1. when the `with` statement is executing: we **enter** the context
2. when the `with` block is done: we **exit** the context

We can create our own context manager using a class that implements an `__enter__` method which is executed when we enter the context, and an `__exit__` method that is executed when we exit the context.

There is a general pattern that context managers can help us deal with:
* Open - Close
* Lock - Release
* Change - Reset
* Enter - Exit
* Start - Stop

In [ ]:
def my_func():
    return 1.0 / 0.0

my_func()

ZeroDivisionError: float division by zero

In [ ]:
class MyContext:
    def __init__(self):
        self.obj = None
        
    def __enter__(self):
        print('entering context...')
        self.obj = 'the Return Object'
        return self.obj

    def __exit__(self, exc_type, exc_value, exc_traceback):
        print('exiting context...')
        if exc_type:
            print(f'*** Error occurred: {exc_type}, {exc_value}')
        return False  # do not suppress exceptions

In [ ]:
with MyContext() as obj:
    raise ValueError


entering context...
exiting context...
*** Error occurred: <class 'ValueError'>, 


ValueError: 

In [ ]:
class MyContext:
    def __init__(self):
        self.obj = None
        
    def __enter__(self):
        print('entering context...')
        self.obj = 'the Return Object'
        return self.obj

    def __exit__(self, exc_type, exc_value, exc_traceback):
        print('exiting context...')
        if exc_type:
            print(f'*** Error occurred: {exc_type}, {exc_value}')
        return True  # suppress exceptions

In [ ]:
with MyContext() as obj:
    raise ValueError
print('reached here without an exception...')

entering context...
exiting context...
*** Error occurred: <class 'ValueError'>, 
reached here without an exception...


In [ ]:
with MyContext() as obj:
    print('running inside with block...')
    i = 'Rohan'
    print(obj)

print(obj)
print(i)

entering context...
running inside with block...
the Return Object
exiting context...
the Return Object
Rohan


In [ ]:
class Resource:
    def __init__(self, name):
        self.name = name
        self.state = None

In [ ]:
class ResourceManager:
    def __init__(self, name):
        self.name = name
        self.resource = None

    def __enter__(self):
        print('entering context')
        self.resource = Resource(self.name)
        self.resource.state = 'created'
        return self.resource

    def __exit__(self, exc_type, exc_value, exc_traceback):
        print('exiting context')
        self.resource.state = 'destroyed'
        if exc_type:
            print('error occurred')
        return False

In [ ]:
with ResourceManager('spam') as res:
    print(f'{res.name} = {res.state}')

print(f'{res.name} = {res.state}')

entering context
spam = created
exiting context
spam = destroyed


In [ ]:
res

In [ ]:
class File:
    def __init__(self, name, mode):
        self.name = name
        self.mode = mode

    def __enter__(self):
        print('opening file...')
        self.file = open(self.name, self.mode)
        self.file.write('This is a new dianosaur')
        return self.file

    def __exit__(self, exc_type, exc_value, exc_traceback):
        print('closing file...')
        self.file.close()
        return False

In [ ]:
with File('test.txt', 'w') as f:
    raise ValueError
    f.write('This is a late dianosaur!')
    

opening file...
closing file...


ValueError: 

In [ ]:
def test():
    with File('test.txt', 'w') as f:
        f.write('This is a late parrot')
        if True:
            return f
        print(f.closed)
    print(f.closed)

In [ ]:
f = test()

opening file...
closing file...


In [ ]:
class File():
    def __init__(self, name, mode):
        self.name = name
        self.mode = mode
        
    def __enter__(self):
        print('opening file...')
        self.file = open(self.name, self.mode)
        return self
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        print('closing file...')
        self.file.close()
        return False

In [ ]:
with File('test.txt', 'r') as file_ctx:
    print(next(file_ctx.file))
    print(file_ctx.name)
    print(file_ctx.mode)

opening file...
This is a new dianosaurThis is a late parrot
test.txt
r
closing file...


In [ ]:
import csv

def read_data():
    with open('nyc_parking_tickets_extract.csv') as f:
        return csv.reader(f, delimiter=',', )

In [ ]:
for row in read_data():
    print(row)

ValueError: I/O operation on closed file.

In [ ]:
# BAD CODE, DO NOT DO THIS UNLESS THIS IS YOUR LAST DAY IN THE COMPANY YOU HATE (PLEASE DONT)
def read_data():
    with open('nyc_parking_tickets_extract.csv') as f:
        return list(csv.reader(f, delimiter=',', quotechar='"'))

for row in read_data():
    print(row)

['Summons Number', 'Plate ID', 'Registration State', 'Plate Type', 'Issue Date', 'Violation Code', 'Vehicle Body Type', 'Vehicle Make', 'Violation Description']
['4006478550', 'VAD7274', 'VA', 'PAS', '10/5/2016', '5', '4D', 'BMW', 'BUS LANE VIOLATION']
['4006462396', '22834JK', 'NY', 'COM', '9/30/2016', '5', 'VAN', 'CHEVR', 'BUS LANE VIOLATION']
['4007117810', '21791MG', 'NY', 'COM', '4/10/2017', '5', 'VAN', 'DODGE', 'BUS LANE VIOLATION']
['4006265037', 'FZX9232', 'NY', 'PAS', '8/23/2016', '5', 'SUBN', 'FORD', 'BUS LANE VIOLATION']
['4006535600', 'N203399C', 'NY', 'OMT', '10/19/2016', '5', 'SUBN', 'FORD', 'BUS LANE VIOLATION']
['4007156700', '92163MG', 'NY', 'COM', '4/13/2017', '5', 'VAN', 'FRUEH', 'BUS LANE VIOLATION']
['4006687989', 'MIQ600', 'SC', 'PAS', '11/21/2016', '5', 'VN', 'HONDA', 'BUS LANE VIOLATION']
['4006943052', '2AE3984', 'MD', 'PAS', '2/1/2017', '5', 'SW', 'LINCO', 'BUS LANE VIOLATION']
['4007306795', 'HLG4926', 'NY', 'PAS', '5/30/2017', '5', 'SUBN', 'TOYOT', 'BUS LANE

In [ ]:
def read_data():
    with open('nyc_parking_tickets_extract.csv') as f:
        yield from csv.reader(f, delimiter=',', quotechar='"')

for row in read_data():
    print(row)

['Summons Number', 'Plate ID', 'Registration State', 'Plate Type', 'Issue Date', 'Violation Code', 'Vehicle Body Type', 'Vehicle Make', 'Violation Description']
['4006478550', 'VAD7274', 'VA', 'PAS', '10/5/2016', '5', '4D', 'BMW', 'BUS LANE VIOLATION']
['4006462396', '22834JK', 'NY', 'COM', '9/30/2016', '5', 'VAN', 'CHEVR', 'BUS LANE VIOLATION']
['4007117810', '21791MG', 'NY', 'COM', '4/10/2017', '5', 'VAN', 'DODGE', 'BUS LANE VIOLATION']
['4006265037', 'FZX9232', 'NY', 'PAS', '8/23/2016', '5', 'SUBN', 'FORD', 'BUS LANE VIOLATION']
['4006535600', 'N203399C', 'NY', 'OMT', '10/19/2016', '5', 'SUBN', 'FORD', 'BUS LANE VIOLATION']
['4007156700', '92163MG', 'NY', 'COM', '4/13/2017', '5', 'VAN', 'FRUEH', 'BUS LANE VIOLATION']
['4006687989', 'MIQ600', 'SC', 'PAS', '11/21/2016', '5', 'VN', 'HONDA', 'BUS LANE VIOLATION']
['4006943052', '2AE3984', 'MD', 'PAS', '2/1/2017', '5', 'SW', 'LINCO', 'BUS LANE VIOLATION']
['4007306795', 'HLG4926', 'NY', 'PAS', '5/30/2017', '5', 'SUBN', 'TOYOT', 'BUS LANE

In [ ]:
f = open('test.txt', 'w')

f.writelines('we are in the context manager class')

f.close()

In [ ]:
with open('test.txt') as f:
    print(f.readlines())

['we are in the context manager class']


In [ ]:
class DataIterator:
    def __init__(self, fname):
        self._fname = fname
        self._f = None
    
    def __iter__(self):
        return self
    
    def __next__(self):
        row = next(self._f)
        return row.strip('\n').split(',')
    
    def __enter__(self):
        self._f = open(self._fname)
        return self
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        if not self._f.closed:
            self._f.close()
        return False

In [ ]:
with DataIterator('nyc_parking_tickets_extract.csv') as data:
    for row in data:
        print(row)

['Summons Number', 'Plate ID', 'Registration State', 'Plate Type', 'Issue Date', 'Violation Code', 'Vehicle Body Type', 'Vehicle Make', 'Violation Description']
['4006478550', 'VAD7274', 'VA', 'PAS', '10/5/2016', '5', '4D', 'BMW', 'BUS LANE VIOLATION']
['4006462396', '22834JK', 'NY', 'COM', '9/30/2016', '5', 'VAN', 'CHEVR', 'BUS LANE VIOLATION']
['4007117810', '21791MG', 'NY', 'COM', '4/10/2017', '5', 'VAN', 'DODGE', 'BUS LANE VIOLATION']
['4006265037', 'FZX9232', 'NY', 'PAS', '8/23/2016', '5', 'SUBN', 'FORD', 'BUS LANE VIOLATION']
['4006535600', 'N203399C', 'NY', 'OMT', '10/19/2016', '5', 'SUBN', 'FORD', 'BUS LANE VIOLATION']
['4007156700', '92163MG', 'NY', 'COM', '4/13/2017', '5', 'VAN', 'FRUEH', 'BUS LANE VIOLATION']
['4006687989', 'MIQ600', 'SC', 'PAS', '11/21/2016', '5', 'VN', 'HONDA', 'BUS LANE VIOLATION']
['4006943052', '2AE3984', 'MD', 'PAS', '2/1/2017', '5', 'SW', 'LINCO', 'BUS LANE VIOLATION']
['4007306795', 'HLG4926', 'NY', 'PAS', '5/30/2017', '5', 'SUBN', 'TOYOT', 'BUS LANE

In [ ]:
data = DataIterator('nyc_parking_tickets_extract.csv')

In [ ]:
for row in data:
    print(row)

TypeError: 'NoneType' object is not an iterator

In [ ]:
data_iter = DataIterator('nyc_parking_tickets_extract.csv')

In [ ]:
with data_iter as data:
    for row in data:
        print(row)

['Summons Number', 'Plate ID', 'Registration State', 'Plate Type', 'Issue Date', 'Violation Code', 'Vehicle Body Type', 'Vehicle Make', 'Violation Description']
['4006478550', 'VAD7274', 'VA', 'PAS', '10/5/2016', '5', '4D', 'BMW', 'BUS LANE VIOLATION']
['4006462396', '22834JK', 'NY', 'COM', '9/30/2016', '5', 'VAN', 'CHEVR', 'BUS LANE VIOLATION']
['4007117810', '21791MG', 'NY', 'COM', '4/10/2017', '5', 'VAN', 'DODGE', 'BUS LANE VIOLATION']
['4006265037', 'FZX9232', 'NY', 'PAS', '8/23/2016', '5', 'SUBN', 'FORD', 'BUS LANE VIOLATION']
['4006535600', 'N203399C', 'NY', 'OMT', '10/19/2016', '5', 'SUBN', 'FORD', 'BUS LANE VIOLATION']
['4007156700', '92163MG', 'NY', 'COM', '4/13/2017', '5', 'VAN', 'FRUEH', 'BUS LANE VIOLATION']
['4006687989', 'MIQ600', 'SC', 'PAS', '11/21/2016', '5', 'VN', 'HONDA', 'BUS LANE VIOLATION']
['4006943052', '2AE3984', 'MD', 'PAS', '2/1/2017', '5', 'SW', 'LINCO', 'BUS LANE VIOLATION']
['4007306795', 'HLG4926', 'NY', 'PAS', '5/30/2017', '5', 'SUBN', 'TOYOT', 'BUS LANE

In [ ]:
import decimal

In [ ]:
decimal.getcontext()

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

In [ ]:
decimal.getcontext().prec=14
decimal.getcontext()

Context(prec=14, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

In [ ]:
decimal.getcontext().prec=28

In [ ]:
old_prec = decimal.getcontext().prec
decimal.getcontext().prec = 4
print(decimal.Decimal(1) / decimal.Decimal(3))
decimal.getcontext().prec = old_prec
print(decimal.Decimal(1) / decimal.Decimal(3))

0.3333
0.3333333333333333333333333333


In [ ]:
class Precision:
    def __init__(self, prec):
        self.prec = prec
        self.current_prec = decimal.getcontext().prec
        
    def __enter__(self):
        decimal.getcontext().prec = self.prec
        
    def __exit__(self, exc_type, exc_value, exc_traceback):
        decimal.getcontext().prec = self.current_prec
        return False      
        

In [ ]:
with Precision(3):
    print(decimal.Decimal(1) / decimal.Decimal(3))
print(decimal.Decimal(1) / decimal.Decimal(3))

0.333
0.3333333333333333333333333333


In [ ]:
import decimal
with decimal.localcontext() as ctx:
    ctx.prec = 3
    print(ctx)
    print(decimal.Decimal(1) / decimal.Decimal(3))
print(decimal.Decimal(1) / decimal.Decimal(3))

Context(prec=3, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])
0.333
0.3333333333333333333333333333


In [ ]:
from time import perf_counter, sleep

class Timer:
    def __init__(self):
        self.elapsed = 0
        
    def __enter__(self):
        self.start = perf_counter()
        return self
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        self.stop = perf_counter()
        self.elapsed = self.stop - self.start
        return False

In [ ]:
with Timer() as timer:
    sleep(3)
print(timer.elapsed)

3.000144999474287


In [ ]:
import sys

class OutToFile:
    def __init__(self, fname):
        self._fname = fname
        self._current_stdout = sys.stdout
        
    def __enter__(self):
        self._file = open(self._fname, 'w')
        print(sys.stdout)
        sys.stdout = self._file
        print("where am i going??", sys.stdout) #THINK MORE
        
    def __exit__(self, exc_type, exc_value, exc_tb):
        sys.stdout = self._current_stdout
        print(self._current_stdout)

        if self._file:
            self._file.close()
        return False
    

In [ ]:
with OutToFile('test.txt'):
    print('Line 1')
    print('Line 2')
print('line 3')


line 3


In [ ]:
class Tag:
    def __init__(self, tag):
        self._tag = tag
        
    def __enter__(self):
        print(f'<{self._tag}>', end='')
        
    def __exit__(self, exc_type, exc_value, exc_tb):
        print(f'</{self._tag}>', end='')
        return False

In [ ]:
with Tag('p'):
    print('The ', end='')
    with Tag('b'):
        print('School ', end='')
    print(' Of AI', end='')

<p>The <b>School </b> Of AI</p>

<p>The <b>School </b> Of AI</p>

- item 1
    - time 1b

In [ ]:
class ListMaker:
    def __init__(self, title, prefix='◼ ', indent=3):
        self._title = title
        self._prefix = prefix
        self._indent = indent
        self._current_indent = 0
        print(title)
        
    def __enter__(self):
        self._current_indent += self._indent
        return self
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        self._current_indent -= self._indent
        return False
        
    def print(self, arg):
        s = ' ' * self._current_indent + self._prefix + str(arg)
        print(s)

In [ ]:
with ListMaker('Items') as lm:
    lm.print('Item 1')
    with lm:
        lm.print('item 1a')
        lm.print('item 1b')
    lm.print('Item 2')
    with lm:
        lm.print('item 2a')
        with lm:
            lm.print('item 2a1')




Items
   ◼ Item 1
      ◼ item 1a
      ◼ item 1b
   ◼ Item 2
      ◼ item 2a
         ◼ item 2a1


In [ ]:
with OutToFile('my_list.txt'):
    with ListMaker('Items') as lm:
        lm.print('Item 1')
        with lm:
            lm.print('item 1a')
            lm.print('item 1b')
        lm.print('Item 2')
        with lm:
            lm.print('item 2a')
            lm.print('item 2b')

In [ ]:
with open('my_list.txt') as f:
    for row in f:
        print(row, end='')

where am i going?? <_io.TextIOWrapper name='my_list.txt' mode='w' encoding='UTF-8'>
Items
   ◼ Item 1
      ◼ item 1a
      ◼ item 1b
   ◼ Item 2
      ◼ item 2a
      ◼ item 2b


In [ ]:
def my_gen():
    try:
        print('creating context and yielding object')
        lst = [1, 2, 3, 4, 5]
        yield lst # << this is equivalent to the __enter__ return
    finally:
        print('exiting context and cleaning up')

In [ ]:
gen = my_gen()

In [ ]:
lst = next (gen)

creating context and yielding object


In [ ]:
print(lst)

[1, 2, 3, 4, 5]


In [ ]:
next (gen)

exiting context and cleaning up


StopIteration: 

In [ ]:
gen = my_gen()
lst = next(gen)
print(lst)
try:
    next(gen)
except StopIteration:
    pass

creating context and yielding object
[1, 2, 3, 4, 5]
exiting context and cleaning up


In [ ]:
class GenCtxManager:
    def __init__(self, gen_func):
        self._gen = gen_func()

    def __enter__(self):
        return next(self._gen)

    def __exit__(self, t, v, tb):
        try:
            next(self._gen)
        except StopIteration:
            pass
        return False

In [ ]:
with GenCtxManager(my_gen) as lst:
    print(lst)

creating context and yielding object
[1, 2, 3, 4, 5]
exiting context and cleaning up


In [ ]:
class GenCtxManager:
    def __init__(self, gen_func, *args, **kwargs):
        self._gen = gen_func(*args, **kwargs)

    def __enter__(self):
        return next(self._gen)

    def __exit__(self, t, v, tb):
        try:
            next(self._gen)
        except StopIteration:
            pass
        return False

In [ ]:
def open_file(fname, mode):
    try:
        print('opening file')
        f = open(fname, mode)
        yield f
    finally:
        print('closing file')
        f.close()

In [ ]:
with GenCtxManager(open_file, 'test.txt', 'w') as f:
    print('writing to the file')
    f.write('testing this file')

opening file
writing to the file
closing file


In [ ]:
with open('test.txt') as f:
    print(next(f))

testing this file


## Using Decorator to Create Context Managers Uusing Generator Functions

In [ ]:
def open_file(fname, mode='r'):
    print('opening file...')
    f = open(fname, mode)
    try:
        yield f
    finally:
        print('closing file...')
        f.close()

In [ ]:
class GenContextManager:
    def __init__(self, gen):
        self.gen = gen
        
    def __enter__(self):
        return next(self.gen)
        
    def __exit__(self, exc_type, exc_value, exc_tb):
        print('calling next to perform cleanup in generator')
        try:
            next(self.gen)
        except StopIteration:
            pass
        return False

In [ ]:
file_gen = open_file('test.txt', 'w')

with GenCtxManager(file_gen) as f:
    f.writelines('Dhoni')

opening file...
calling next to perform cleanup in generator
closing file...


In [ ]:
file_gen = open_file('test.txt')
with GenContextManager(file_gen) as f:
    print(f.readlines())

opening file...
['Dhoni']
calling next to perform cleanup in generator
closing file...


In [ ]:
def context_manager_dec(gen_fn):
    def helper(*args, **kwargs):
        gen = gen_fn(*args, **kwargs)
        ctx = GenContextManager(gen)
        return ctx
    return helper

In [ ]:
# open_file = context_manager_dec(open_file)
@context_manager_dec
def open_file(fname, mode='r'):
    print('opening file...')
    f = open(fname, mode)
    try:
        yield f
    finally:
        print('closing file...')
        f.close()

In [ ]:
with open_file('test.txt') as f:
    print(f.readline())

opening file...
Dhoni
calling next to perform cleanup in generator
closing file...


In [ ]:
from contextlib import contextmanager

@contextmanager
def open_file(fname, mode='r'):
    print('opening file...')
    f = open(fname, mode)
    try:
        yield f
    finally:
        print('closing file...')
        f.close()

In [ ]:
with open_file('test.txt') as f:
    print(f.readlines())

opening file...
['Dhoni']
closing file...


In [ ]:
from time import perf_counter, sleep

In [ ]:
@contextmanager
def timer():
    stats = dict()
    start = perf_counter()
    stats['start'] = start
    try:
        yield stats
    finally:
        end = perf_counter()
        stats['end'] = end
        stats['elapsed'] = end - start

In [ ]:
with timer() as stats:
    sleep(1)

In [ ]:
print(stats)

{'start': 6198162.101008153, 'end': 6198163.101815342, 'elapsed': 1.0008071893826127}


In [ ]:
import sys

@contextmanager
def out_to_file(fname):
    current_stdout = sys.stdout
    file = open(fname, 'w')
    sys.stdout = file
    try:
        yield None
    finally:
        file.close()
        sys.stdout = current_stdout

In [ ]:
with out_to_file('test.txt'):
    print('line 1')
    print('line 2')

In [ ]:
with open('test.txt') as f:
    print(f.readlines())

['line 1\n', 'line 2\n']


In [ ]:
print('line 1')

line 1


In [ ]:
from contextlib import redirect_stdout

In [ ]:
with open('test.txt', 'w') as f:
    with redirect_stdout(f):
        print('Look on the bright side of life')

In [ ]:
with open('test.txt') as f:
    print(f.readlines())

['Look on the bright side of life\n']


# 1st Assignment